In [1]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Subset
from urllib.parse import urlparse
from three_datasets import all_dataset
import random
tqdm.pandas()
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"✅ Seed fixed to {seed}")

  label                                                url
0     2                        https://blog.sockpuppet.us/
1     2                  https://blog.apiki.com/seguranca/
2     1  http://autoecole-lauriston.com/a/T0RVd056QXlNe...
3     1  http://chinpay.site/index.html?hgcFSE@E$Z*DFcG...
4     2  http://www.firstfivenebraska.org/blog/article/...


c:\Users\rrpra\Documents\Github\git_testing\email_and_url_classifier_using_nlp_feature_extraction_sem_5_project\three_datasets.py:70: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df4['label'][df4['label'] == 1] = 0
c:\Users\rrpra\Documents\


✅ Dataset 1 (Malicious URLs) cleaned: 522214 samples (Phishing=94111, Benign=428103)

📂 Dataset 1 (Malicious URLs) split → Train: 417732, Valid: 52217, Test: 52217

📂 Dataset 1 (Malicious URLs) split → Train: 188296, Valid: 23537, Test: 23537

📂 Dataset 1 (Malicious URLs) split → Train: 528097, Valid: 66012, Test: 66013

📂 Dataset 1 (Malicious URLs) split → Train: 639999, Valid: 80000, Test: 80000
✅ Seed fixed to 42


In [ ]:
from transformers import AutoTokenizer,  T5EncoderModel
import torch
import numpy as np

model_name = "google/canine-s"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5EncoderModel.from_pretrained(model_name)  
model.eval()
def get_byt5_embedding(url):
    # Tokenize the input
    inputs = tokenizer(url, return_tensors="pt", truncation=True, padding=True, max_length=50)
    
    # Get encoder output (not generate)
    with torch.no_grad():
        outputs = model(**inputs)
        # outputs.last_hidden_state: [batch_size, seq_len, hidden_dim]
         
        # Mean pooling across sequence
        emb = outputs.last_hidden_state.mean(dim=1).reshape(-1)
    return emb.cpu().numpy()

# Example
#url = "http://paypal-login-secure-update.com"
#vector = get_byt5_embedding(url)
#print("Embedding shape:", vector.shape)  # e.g. (512,) or (768,) depending on model size


c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load once globally (so it doesn't reload every call)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "google/canine-s"  # or "google/canine-c" (larger, slower)

# Load model + tokenizer only once
tokenizer_canine = AutoTokenizer.from_pretrained(MODEL_NAME)
model_canine = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
model_canine.eval()

@torch.no_grad()
def get_canine_embedding(url, max_length=50, pooling="mean"):
    """
    Returns a fixed-length CANINE embedding for a single URL string.

    Args:
        url (str): URL text to encode.
        max_length (int): Maximum length of character sequence (truncate/pad to this).
        pooling (str): 'mean' or 'max' for pooling token embeddings into one vector.

    Returns:
        np.ndarray: 1D embedding vector (shape [hidden_dim], e.g. [768]).
    """
    import numpy as np

    # Tokenize
    encoded = tokenizer_canine(
        url,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    ).to(DEVICE)

    # Forward pass
    outputs = model_canine(**encoded)
    hidden = outputs.last_hidden_state  # [1, seq_len, hidden_dim]
    mask = encoded["attention_mask"].unsqueeze(-1).type_as(hidden)  # [1, seq_len, 1]

    # Pooling
    if pooling == "mean":
        summed = (hidden * mask).sum(dim=1)
        denom = mask.sum(dim=1).clamp(min=1e-9)
        pooled = summed / denom  # [1, hidden_dim]
    elif pooling == "max":
        hidden = hidden.masked_fill(mask == 0, -1e9)
        pooled = hidden.max(dim=1).values  # [1, hidden_dim]
    else:
        raise ValueError("Pooling must be 'mean' or 'max'")

    return pooled.squeeze(0).cpu().numpy()  # (hidden_dim,)


c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

In [3]:
# ============================================================
# Character Encoding Setup
# ============================================================

# Allowed printable ASCII chars
ascii_chars = [chr(i) for i in range(32, 127)]

# Special control tokens
special_tokens = [
    '<PAD>', '<UNK>',
]

# Build vocab and mapping
vocab = special_tokens + ascii_chars
char2idx = {ch: i for i, ch in enumerate(vocab)}



def encode(text, max_len=100):
    indices = torch.full((max_len,), char2idx["<PAD>"], dtype=torch.long)
    text = text.lower()[:max_len]
    for i, c in enumerate(text):
        indices[i] = char2idx.get(c, char2idx["<UNK>"])

    return indices





encoded_data = {}
frac = 1
gen = all_dataset()



for i, splits in zip([f'dataset_{i}' for i in range(4)], gen):
    encoded_data[i] = {}
    print(f"\n🔠 Encoding structured URLs for {i}...")

    for split_name, df in zip(['train', 'valid', 'test'], splits):
        df = df.sample(frac=frac, random_state=42)
        df["encode"] = df["url"].progress_apply(encode)
        encoded_data[i][split_name] = df
        print(f"  ✅ {split_name}: Encoded {len(df)} URLs")
    next(gen)
    next(gen)
    next(gen)

print("\n✅ All datasets encoded with proper start/end markers and padding!")


🔠 Encoding structured URLs for dataset_0...


100%|██████████| 417732/417732 [01:16<00:00, 5494.38it/s]


  ✅ train: Encoded 417732 URLs


100%|██████████| 52217/52217 [00:10<00:00, 5121.80it/s]


  ✅ valid: Encoded 52217 URLs


100%|██████████| 52217/52217 [00:09<00:00, 5490.47it/s]

  ✅ test: Encoded 52217 URLs

✅ All datasets encoded with proper start/end markers and padding!


In [4]:
from torch.utils.data import TensorDataset, DataLoader
# ============================================================
# Convert to TensorDataset and DataLoader
# ============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128

dataloader_dict = {}

def make_tensor_dataset(df):
    url_tensor = torch.stack(list(df["encode"]))
    labels_tensor = torch.tensor(df["label"].values, dtype=torch.long)
    return TensorDataset(url_tensor, labels_tensor)

for name, splits in encoded_data.items():
    dataloader_dict[name] = {}
    print(f"\n📦 Creating DataLoaders for {name}...")
    
    train_set = make_tensor_dataset(splits["train"])
    val_set = make_tensor_dataset(splits["valid"])
    test_set = make_tensor_dataset(splits["test"])
    
    dataloader_dict[name]["train_loader"] = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    dataloader_dict[name]["val_loader"] = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    dataloader_dict[name]["test_loader"] = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    print(f"✅ DataLoaders ready for {name} (Train/Val/Test)")

print("\n🚀 All DataLoaders are ready in `dataloader_dict`!")

# Example Access:
# dataloader_dict["dataset1"]["train_loader"]



📦 Creating DataLoaders for dataset_0...
✅ DataLoaders ready for dataset_0 (Train/Val/Test)

🚀 All DataLoaders are ready in `dataloader_dict`!


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# =====================================================
# 🔹 TinyByT5 Encoder (Reduced depth for short sequences)
# =====================================================
class TinyByT5Encoder(nn.Module):
    def __init__(self,
                 vocab_size=256,
                 d_model=128,
                 num_layers=2,
                 num_heads=2,
                 dim_ff=256,
                 max_len=100,
                 n_out=128,
                 dropout=0.1):
        super().__init__()

        # 🔹 Byte embedding layer (0–255)
        self.embedding = nn.Embedding(vocab_size, d_model)

        # 🔹 Positional embeddings
        self.pos_embedding = nn.Embedding(max_len, d_model)

        # 🔹 Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=dim_ff,
            dropout=dropout,
            activation="gelu",
            batch_first=True,
            norm_first=True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 🔹 Final normalization
        self.final_norm = nn.LayerNorm(d_model)
        self.projection = nn.Linear(in_features=d_model, out_features=n_out)

    def forward(self, x):
        """
        x: (batch, seq_len) — byte indices [0–255]
        """
        batch_size, seq_len = x.size()
        device = x.device

        positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, -1)
        x = self.embedding(x) + self.pos_embedding(positions)

        x = self.encoder(x)
        x = self.projection(self.final_norm(x))

        return x  # (B, L, d_model)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================
# 🔹 Tiny Transformer Encoder
# ==========================
class TinyURLTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=2, num_layers=2, ff_dim=128, max_len=100, out_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            batch_first=True,
            activation="gelu"
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.projection = nn.Linear(embed_dim, out_dim)

    def forward(self, x):
        # x: (batch, seq_len)
        #x = self.embedding(x) + self.pos_embedding[:, :x.size(1), :]  # (B, L, E)
        #x = self.transformer(x)                                       # (B, L, E)
        return self.embedding(x) #self.projection(x)                                     # (B, L, out_dim)

class URLBinaryCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, maxlen=100):
        super(URLBinaryCNN, self).__init__()
        self.maxlen = maxlen

        self.transformer = TinyByT5Encoder(
            vocab_size=vocab_size,
            max_len=maxlen,
            d_model=512,
            n_out=embed_dim
        )
        # 🔹 1st Conv block
        self.conv1_3x3 = nn.Conv1d(in_channels=embed_dim, out_channels=32, kernel_size=3, padding=1)
        self.convreduce1_1x1 = nn.Conv1d(in_channels=embed_dim, out_channels=1, kernel_size=1, padding=0)
        self.conv1_5x5 = nn.Conv1d(in_channels=embed_dim, out_channels=32, kernel_size=5, padding=2)
        self.conv1_7 = nn.Conv1d(in_channels=embed_dim, out_channels=32, kernel_size=7, padding=3)
        self.conv1_1x1 = nn.Conv1d(in_channels=32*3, out_channels=32, kernel_size=1, padding=0)
        self.layer_norm1 = nn.LayerNorm(normalized_shape=[32, maxlen])


         # 🔹 2st Conv block
        self.conv2_3x3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.convreduce2_1x1 = nn.Conv1d(in_channels=128, out_channels=1, kernel_size=3, padding=1)
        self.conv2_5x5 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=5, padding=2)
        self.conv2_1x1 = nn.Conv1d(in_channels=64*2, out_channels=64, kernel_size=3, padding=1)
        self.layer_norm2 = nn.LayerNorm(normalized_shape=[64, maxlen])
        

         # 🔹 3st Conv block
        self.conv3_3x3 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.convreduce3_1x1 = nn.Conv1d(in_channels=64, out_channels=1, kernel_size=1, padding=0)
        self.conv3_5x5 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=5, padding=2)
        self.conv3_7 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=7, padding=3)
        self.conv3_1x1 = nn.Conv1d(in_channels=32*3, out_channels=32, kernel_size=1, padding=0)
        self.layer_norm3 = nn.LayerNorm(normalized_shape=[32, maxlen])

        self.lstm = nn.LSTM(input_size=32, hidden_size=32, num_layers=1, 
                            batch_first=True)
        
        self.aap = nn.AdaptiveAvgPool1d(1)

        # 🔹 Fully connected layers

        self.flatten = nn.Flatten()
        self.fc_layers = nn.Sequential(
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(0.5),



            nn.Linear(16, 1)
        )
    def forward(self, x):
        # (batch, seq_len)
        x = self.transformer(x)           # (B, L, E)
        
        x = x.permute(0, 2, 1)            # (B, E, L)



        # 🔹 Block 1
        x3 = F.relu(self.conv1_3x3(x))

        #xr = F.relu(self.convreduce1_1x1(x))
        x5 = F.relu(self.conv1_5x5(x))
        x7 = F.relu(self.conv1_7(x))

        x = torch.cat([x3, x5, x7], dim=1)
        #print(x.shape)
        x = F.relu(self.conv1_1x1(x))

        #x = self.layer_norm1(x)


        '''
        # 🔹 Block 2
        x3 = F.relu(self.conv2_3x3(x))
        xr = F.relu(self.convreduce2_1x1(x))
        x5 = F.relu(self.conv2_5x5(xr))
        x = torch.cat([x3, x5], dim=1)
        x = F.relu(self.conv2_1x1(x))
        x = self.layer_norm2(x)
        '''
        # 🔹 Block 3
        #x = F.relu(self.conv3_3x3(x))
        #xr = F.relu(self.convreduce3_1x1(x))
        #x5 = F.relu(self.conv3_5x5(x))
        #x7 = F.relu(self.conv3_7(x))
        #x = torch.cat([x3, x5, x7], dim=1)
        #x = F.relu(self.conv3_1x1(x))
        #x = self.layer_norm3(x)

        x = x.permute(0, 2, 1)

        x, _ = self.lstm(x)
        x = x[:, -1, :]
        # 🔹 Global Average Pooling + FC
        #x = self.aap(x)                   # (batch, channels, 1)
        x = self.flatten(x)               # (batch, channels)
        x = self.fc_layers(x)

        return torch.sigmoid(x)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import sys

class Train:
    def __init__(self, 
                 model, 
                 criterion, 
                 transformer_optimizer=None,
                 main_optimizer = None, 
                 scheduler_t=None,
                 scheduler_c=None,
                 train_loader=None, 
                 val_loader=None,   
                 device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        
        """
        optimizer_groups: dict with keys like {"transformer": optimizer1, "cnn": optimizer2}
        schedulers: dict with keys matching optimizer_groups (optional)
        """
        self.model = model
        self.criterion = criterion

        self.transformer_params = []
        self.cnn_params = []
        for name, param in model.named_parameters():
            if name.startswith("transformer."):
                self.transformer_params.append(param)
            else:
                self.cnn_params.append(param)



        self.transformer_optimizer = optim.Adam(self.transformer_params, lr=1e-4) if transformer_optimizer is None  else transformer_optimizer
        self.main_optimizer = optim.Adam(self.cnn_params, lr=1e-3) if main_optimizer is None else main_optimizer
        self.scheduler_t = optim.lr_scheduler.ReduceLROnPlateau(self.transformer_optimizer, mode='min', factor=0.5, patience=2) if scheduler_t is None  else scheduler_t
        self.scheduler_c = optim.lr_scheduler.ReduceLROnPlateau(self.main_optimizer, mode='min', factor=0.5, patience=2) if scheduler_c is None  else scheduler_c
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.train_losses, self.val_losses = [], []
        self.train_accs, self.val_accs = [], []


    def freeze_module(self, module):
        for param in module.parameters():
            param.requires_grad = False

    def unfreeze_module(self, module):
        for param in module.parameters():
            param.requires_grad = True


    def train(self, epochs_list=[3,3,4], early_stopping=True, frac=1.0, val_frac=1.0, alt_cycle = 2,  log=False):
        for phase, epochs in enumerate(epochs_list):
            for epoch in range(epochs):
                if phase == 0:
                    # 🧠 Train Transformer — freeze CNN/LSTM layers
                    self.unfreeze_module(self.model.transformer)
                    for name, module in self.model.named_children():
                        if name != "transformer":
                            self.freeze_module(module)
                        else:
                            self.unfreeze_module(module)
                    active_optims = [self.transformer_optimizer]
                    active_scheds = [self.scheduler_t]
                    phase_name = "Transformer"
                elif phase == 1:
                    # 🎯 Train CNN/LSTM/FC — freeze Transformer
                    self.freeze_module(self.model.transformer)
                    for name, module in self.model.named_children():
                        if name == "transformer":
                            self.freeze_module(module)
                        else:
                            self.unfreeze_module(module)
                    active_optims = [self.main_optimizer]
                    active_scheds = [self.scheduler_c]
                    phase_name = "CNN"
                else:
                    self.freeze_module(self.model.transformer)
                    for name, module in self.model.named_children():
                        self.unfreeze_module(module)
                    active_optims = [self.transformer_optimizer, self.main_optimizer]
                    active_scheds = [self.scheduler_t, self.scheduler_c]
                    phase_name = "CNN + transformer"



                self.model.train()
                train_loss, correct_train, total_train = 0, 0, 0
                max_batches = int(len(self.train_loader) * frac)
                
                for batch_idx, (batch_x, batch_y) in enumerate(self.train_loader):
                    if batch_idx >= max_batches:
                        break
                    
                    batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)
                    
                    for opt in active_optims:
                        opt.zero_grad()

                    outputs = self.model(batch_x)
                    loss = self.criterion(outputs, batch_y)
                    loss.backward()

                    for opt in active_optims:
                        opt.step()




                    # === Metrics ===
                    batch_loss = loss.item()
                    preds = (outputs >= 0.5).float()
                    batch_acc = (preds == batch_y).float().mean().item()

                    train_loss += batch_loss * batch_x.size(0)
                    correct_train += (preds == batch_y).sum().item()
                    total_train += batch_x.size(0)

                    if log and (batch_idx + 1) % 20 == 0:
                        print(f"\rEpoch {epoch+1}/{epochs}: Training {phase_name} | "
                            f"Batch {batch_idx+1}/{max_batches} | "
                            f"Loss: {batch_loss:.4f}, Acc: {batch_acc:.4f}", end='')

                avg_train_loss = train_loss / total_train
                train_acc = correct_train / total_train
                self.train_losses.append(avg_train_loss)
                self.train_accs.append(train_acc)

                # === Validation ===
                if self.val_loader is not None:
                    avg_val_loss, val_acc = self.evaluate(val_frac)
                    self.val_losses.append(avg_val_loss)
                    self.val_accs.append(val_acc)

                    if log:
                        print(f"\rEpoch {epoch+1}/{epochs} Training {phase_name}| "
                            f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
                            f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")
                elif log:
                    print(f"\rEpoch {epoch+1}/{epochs} Training {phase_name}| "
                        f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}")


                for sched in active_scheds:
                    sched.step(avg_val_loss)



        def evaluate(self, frac=1.0):
            self.model.eval()
            val_loss, correct_val, total_val = 0, 0, 0
            max_batches = max(int(len(self.val_loader) * frac), 1)
            print(f'\r total validation batch size {max_batches}'.ljust(100), end='')
            with torch.no_grad():
                for batch_idx, (batch_x, batch_y) in enumerate(self.val_loader):
                    if batch_idx >= max_batches:
                        break
                    batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)
                    outputs = self.model(batch_x)
                    loss = self.criterion(outputs, batch_y)
                    avg_batch_loss = loss.item()
                    val_loss += avg_batch_loss * batch_x.size(0)
                    preds = (outputs >= 0.5).float()
                    correct_val += (preds == batch_y).sum().item()
                    total_val += batch_x.size(0)

            avg_val_loss = val_loss / total_val
            val_acc = correct_val / total_val
            return avg_val_loss, val_acc


In [15]:
# Initialize model, loss, optimizer
model = URLBinaryCNN(vocab_size=len(vocab)).to(device)
transformer_params = []
cnn_params = []
for name, module in model.named_children():
    if name == "transformer":  # all encoder weights
        transformer_params.append(name)
    else:  # everything else (cnn, lstm, fc, etc.)
        cnn_params.append(name)
print(transformer_params)

['transformer']


In [29]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from lion_pytorch import Lion
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
# 🔧 Training Config
# ============================================================
num_epochs = 10
lr = 0.001

# Store all dataset metrics
all_results = {}

# ============================================================
# 🔁 Training Loop for Each Dataset
# ============================================================
for dataset_name, loaders in {'dataset_0':dataloader_dict['dataset_0']}.items():
    print("\n" + "="*70)
    print(f"🚀 Training model on {dataset_name.upper()} dataset")
    print("="*70)
    for frac in  [0.5, 1.0]:
        print(f"🧩 Using {frac*100:.0f}% of training data".center(50, '_'))
        train_loader = loaders["train_loader"]
        val_loader = loaders["val_loader"]

        # Initialize model, loss, optimizer
        model = URLBinaryCNN(vocab_size=len(vocab)).to(device)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=lr/10)
        trainer = Train(model, criterion, train_loader=train_loader, val_loader=val_loader)
        # Lists to track performance
        trainer.train(num_epochs,lr,frac=frac,val_frac=frac, log=True)

    

    # Store all results for this dataset
    all_results[dataset_name] = {
        "train_losses": trainer.train_losses,
        "val_losses": trainer.val_losses,
        "train_accs": trainer.train_accs,
        "val_accs": trainer.val_accs,
        "final_val_acc": trainer.val_accs[-1],
        "final_val_loss": trainer.val_losses[-1]
    }

print("\n✅ All datasets trained successfully!")

# ============================================================
# 📊 Summary of All Results
# ============================================================
print("\n" + "="*70)
print("📈 Final Validation Accuracy Summary")
print("="*70)
for name, res in all_results.items():
    print(f"{name:<20} | Val Acc: {res['final_val_acc']:.4f} | Val Loss: {res['final_val_loss']:.4f}")



🚀 Training model on DATASET_0 dataset
___________🧩 Using 50% of training data___________
Epoch 1/10 Training Transformer| Train Loss: 0.6369, Train Acc: 0.8195 | Val Loss: 0.6318, Val Acc: 0.8200
Epoch 2/10 Training Transformer| Train Loss: 0.6163, Train Acc: 0.8630 | Val Loss: 0.6028, Val Acc: 0.9468
Epoch 3/10 Training CNN| Train Loss: 0.2309, Train Acc: 0.9438 | Val Loss: 0.1833, Val Acc: 0.9475 
Epoch 4/10 Training CNN| Train Loss: 0.1942, Train Acc: 0.9443 | Val Loss: 0.1611, Val Acc: 0.9467 
Epoch 5/10 Training Transformer| Train Loss: 0.1821, Train Acc: 0.9406 | Val Loss: 0.1545, Val Acc: 0.9438
Epoch 6/10 Training Transformer| Train Loss: 0.1687, Train Acc: 0.9431 | Val Loss: 0.1401, Val Acc: 0.9492
Epoch 7/10 Training CNN| Train Loss: 0.1597, Train Acc: 0.9481 | Val Loss: 0.1348, Val Acc: 0.9476 
Epoch 8/10 Training CNN| Train Loss: 0.1555, Train Acc: 0.9513 | Val Loss: 0.1291, Val Acc: 0.9580 
Epoch 9/10 Training CNN + transformer| Train Loss: 0.1476, Train Acc: 0.9539 | Val

In [ ]:

# ============================================================
# 🔁 Training Loop for Each Dataset
# ============================================================
for dataset_name, loaders in {'dataset_0':dataloader_dict['dataset_0']}.items():
    print("\n" + "="*70)
    print(f"🚀 Training model on {dataset_name.upper()} dataset")
    print("="*70)
    for frac in  [0.5, 1.0]:
        print(f"🧩 Using {frac*100:.0f}% of training data".center(50, '_'))
        train_loader = loaders["train_loader"]
        val_loader = loaders["val_loader"]

        # Initialize model, loss, optimizer
        model = URLBinaryCNN(vocab_size=len(vocab)).to(device)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=lr/10)
        trainer = Train(model, criterion, optimizer, train_loader, val_loader)
        # Lists to track performance
        trainer.train(num_epochs,lr,frac=frac,val_frac=frac, log=True)

    

    # Store all results for this dataset
    all_results[dataset_name] = {
        "train_losses": trainer.train_losses,
        "val_losses": trainer.val_losses,
        "train_accs": trainer.train_accs,
        "val_accs": trainer.val_accs,
        "final_val_acc": trainer.val_accs[-1],
        "final_val_loss": trainer.val_losses[-1]
    }

print("\n✅ All datasets trained successfully!")

# ============================================================
# 📊 Summary of All Results
# ============================================================
print("\n" + "="*70)
print("📈 Final Validation Accuracy Summary")
print("="*70)
for name, res in all_results.items():
    print(f"{name:<20} | Val Acc: {res['final_val_acc']:.4f} | Val Loss: {res['final_val_loss']:.4f}")



🚀 Training model on DATASET_0 dataset
___________🧩 Using 10% of training data___________
Epoch 1/6 | Train Loss: 0.2614, Train Acc: 0.9133 | Val Loss: 0.0198, Val Acc: 0.9978              
Epoch 2/6 | Train Loss: 0.1030, Train Acc: 0.9702 | Val Loss: 0.0159, Val Acc: 0.9978              
Epoch 3/6 | Train Loss: 0.0480, Train Acc: 0.9870 | Val Loss: 0.0151, Val Acc: 0.9978              
Epoch 4/6 | Train Loss: 0.0409, Train Acc: 0.9914 | Val Loss: 0.0154, Val Acc: 0.9974              
Epoch 5/6 | Train Loss: 0.0443, Train Acc: 0.9895 | Val Loss: 0.0148, Val Acc: 0.9978              
Epoch 6/6 | Train Loss: 0.0383, Train Acc: 0.9912 | Val Loss: 0.0150, Val Acc: 0.9978              
___________🧩 Using 50% of training data___________
Epoch 1/6 | Train Loss: 0.0871, Train Acc: 0.9817 | Val Loss: 0.0160, Val Acc: 0.9975              
Epoch 2/6 | Train Loss: 0.0614, Train Acc: 0.9895 | Val Loss: 0.0132, Val Acc: 0.9980              
Epoch 3/6 | Train Loss: 0.0356, Train Acc: 0.9935 | Val Los

KeyboardInterrupt: 


======================================================================  
🚀 Training model on DATASET_2 dataset
======================================================================  
___________🧩 Using 10% of training data___________  
Epoch 1/6 | Train Loss: 0.6624, Train Acc: 0.6061 | Val Loss: 0.5799, Val Acc: 0.7219                
Epoch 2/6 | Train Loss: 0.5179, Train Acc: 0.7594 | Val Loss: 0.5534, Val Acc: 0.7312                
Epoch 3/6 | Train Loss: 0.4341, Train Acc: 0.8096 | Val Loss: 0.4320, Val Acc: 0.8120                
Epoch 4/6 | Train Loss: 0.3945, Train Acc: 0.8308 | Val Loss: 0.4003, Val Acc: 0.8252              
Epoch 5/6 | Train Loss: 0.3716, Train Acc: 0.8404 | Val Loss: 0.3567, Val Acc: 0.8496              
Epoch 6/6 | Train Loss: 0.3588, Train Acc: 0.8458 | Val Loss: 0.3478, Val Acc: 0.8523              
___________🧩 Using 50% of training data___________  
Epoch 1/6 | Train Loss: 0.5106, Train Acc: 0.7353 | Val Loss: 0.5287, Val Acc: 0.6781              
Epoch 2/6 | Train Loss: 0.3578, Train Acc: 0.8452 | Val Loss: 0.5693, Val Acc: 0.6529              
Epoch 3/6 | Train Loss: 0.3324, Train Acc: 0.8569 | Val Loss: 0.5073, Val Acc: 0.7133              
Epoch 4/6 | Train Loss: 0.3191, Train Acc: 0.8620 | Val Loss: 0.5989, Val Acc: 0.6296              
Epoch 5/6 | Train Loss: 0.3086, Train Acc: 0.8663 | Val Loss: 0.4869, Val Acc: 0.7649              
Epoch 6/6 | Train Loss: 0.3027, Train Acc: 0.8693 | Val Loss: 0.4661, Val Acc: 0.7953              
__________🧩 Using 100% of training data___________  
Epoch 1/6 | Train Loss: 0.4213, Train Acc: 0.8050 | Val Loss: 0.3473, Val Acc: 0.8572              
Epoch 2/6 | Train Loss: 0.3203, Train Acc: 0.8626 | Val Loss: 0.3535, Val Acc: 0.8393              
Epoch 3/6 | Train Loss: 0.3016, Train Acc: 0.8692 | Val Loss: 0.4240, Val Acc: 0.7920              
Epoch 4/6 | Train Loss: 0.2923, Train Acc: 0.8723 | Val Loss: 0.3619, Val Acc: 0.8312              
Epoch 5/6 | Train Loss: 0.2845, Train Acc: 0.8753 | Val Loss: 0.3901, Val Acc: 0.8187              
Epoch 6/6 | Train Loss: 0.2788, Train Acc: 0.8772 | Val Loss: 0.3979, Val Acc: 0.8223              

✅ All datasets trained successfully!  

======================================================================  
📈 Final Validation Accuracy Summary  
======================================================================  
dataset_2            | Val Acc: 0.8223 | Val Loss: 0.3979  

In [7]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import torch


In [8]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for i in range(0, len(test_url_tensor), 64):  # batch size 64
        batch_x = test_url_tensor[i:i+64].to(device)
        batch_y = test_labels_tensor[i:i+64].to(device)
        
        outputs = model(batch_x)
        preds = (outputs >= 0.5).long().squeeze(1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())


NameError: name 'test_url_tensor' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_BiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim=128):
        super(CNN_BiLSTM, self).__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        
        # Conv blocks
        self.conv1 = nn.Conv1d(embed_dim, 256, kernel_size=8, padding=4)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(256, 128, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool1d(2)
        
        # BiLSTM
        self.lstm = nn.LSTM(input_size=128, hidden_size=64, num_layers=1,
                            batch_first=True, bidirectional=True)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64*2, 128)  # 64*2 because bidirectional
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)  # binary output
        
    def forward(self, x):
        # x: (batch_size, seq_len)
        x = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        x = x.permute(0, 2, 1)  # (batch_size, embed_dim, seq_len) for Conv1d
        
        # Conv block 1
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        # Conv block 2
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        # Prepare for LSTM: (batch_size, seq_len, features)
        x = x.permute(0, 2, 1)
        
        # BiLSTM
        lstm_out, _ = self.lstm(x)  # lstm_out: (batch_size, seq_len, 2*hidden_size)
        
        # Take the last timestep
        x = lstm_out[:, -1, :]  # (batch_size, 128)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.fc2(x)
        
        return x


In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
# 🔧 Training Config
# ============================================================
num_epochs = 3
lr = 0.001
batch_print_interval = 1

# Dictionary to store all dataset results
all_results = {}

# ============================================================
# 🔁 Loop Over Each Dataset
# ============================================================
for dataset_name, loaders in dataloader_dict.items():
    print("\n" + "="*70)
    print(f"🚀 Training CNN-BiLSTM on {dataset_name.upper()} dataset")
    print("="*70)

    train_loader = loaders["train_loader"]
    val_loader = loaders["val_loader"]

    model = CNN_BiLSTM(vocab_size=len(vocab)).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Track metrics
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        correct_train = 0
        total_train = 0

        for batch_idx, (batch_x, batch_y) in enumerate(train_loader):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device).float().unsqueeze(1)
            batch_y = torch.clamp(batch_y, 0, 1)

            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            preds = (torch.sigmoid(outputs) >= 0.5).float()
            acc = (preds == batch_y).float().mean().item()

            train_loss += loss.item() * batch_x.size(0)
            correct_train += (preds == batch_y).sum().item()
            total_train += batch_x.size(0)

            if (batch_idx + 1) % batch_print_interval == 0:
                sys.stdout.write(f"\r[{dataset_name}] Epoch {epoch+1}/{num_epochs} | "
                                 f"Batch {batch_idx+1}/{len(train_loader)} | "
                                 f"Loss: {loss.item():.4f}, Acc: {acc:.4f}")
                sys.stdout.flush()

        avg_train_loss = train_loss / total_train
        train_acc = correct_train / total_train
        print()  # newline

        # === Validation ===
        model.eval()
        val_loss = 0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device).float().unsqueeze(1)
                batch_y = torch.clamp(batch_y, 0, 1)

                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item() * batch_x.size(0)

                preds = (torch.sigmoid(outputs) >= 0.5).float()
                correct_val += (preds == batch_y).sum().item()
                total_val += batch_x.size(0)

        avg_val_loss = val_loss / total_val
        val_acc = correct_val / total_val

        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        print(f"[{dataset_name}] Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # === Save results for this dataset ===
    all_results[dataset_name] = {
        "train_losses": train_losses,
        "val_losses": val_losses,
        "train_accs": train_accs,
        "val_accs": val_accs,
        "final_val_acc": val_accs[-1],
        "final_val_loss": val_losses[-1]
    }

print("\n✅ All datasets trained successfully!")

# ============================================================
# 📊 Summary of Final Results
# ============================================================
print("\n" + "="*70)
print("📈 Final Validation Summary (CNN-BiLSTM)")
print("="*70)
for name, res in all_results.items():
    print(f"{name:<20} | Val Acc: {res['final_val_acc']:.4f} | Val Loss: {res['final_val_loss']:.4f}")



🚀 Training CNN-BiLSTM on DATASET1 dataset
[dataset1] Epoch 1/3 | Batch 102/102 | Loss: 0.2697, Acc: 0.8942
[dataset1] Epoch 1/3 | Train Loss: 0.4063, Train Acc: 0.8384 | Val Loss: 0.2719, Val Acc: 0.8974
[dataset1] Epoch 2/3 | Batch 102/102 | Loss: 0.1222, Acc: 0.9658
[dataset1] Epoch 2/3 | Train Loss: 0.1782, Train Acc: 0.9378 | Val Loss: 0.1227, Val Acc: 0.9609
[dataset1] Epoch 3/3 | Batch 102/102 | Loss: 0.0869, Acc: 0.9730
[dataset1] Epoch 3/3 | Train Loss: 0.1033, Train Acc: 0.9677 | Val Loss: 0.0860, Val Acc: 0.9731

🚀 Training CNN-BiLSTM on DATASET2 dataset
[dataset2] Epoch 1/3 | Batch 47/47 | Loss: 0.0809, Acc: 0.9864
[dataset2] Epoch 1/3 | Train Loss: 0.3467, Train Acc: 0.8455 | Val Loss: 0.0146, Val Acc: 0.9977
[dataset2] Epoch 2/3 | Batch 47/47 | Loss: 0.0022, Acc: 1.0000
[dataset2] Epoch 2/3 | Train Loss: 0.0140, Train Acc: 0.9977 | Val Loss: 0.0102, Val Acc: 0.9981
[dataset2] Epoch 3/3 | Batch 47/47 | Loss: 0.0022, Acc: 1.0000
[dataset2] Epoch 3/3 | Train Loss: 0.0122, Tr